In [0]:
USE CATALOG claims360_dev;

-- Alert: Overall FPY (latest mature month) < target
WITH params AS (
  SELECT 0.85 AS fpy_target
),
latest AS (
  SELECT MAX(to_date(concat(submission_month, '-01'))) AS dt
  FROM gold.v_first_pass_yield
),
agg AS (
  SELECT
    SUM(fpy_claims)       AS fpy_claims_mo,
    SUM(submitted_claims) AS submitted_claims_mo
  FROM gold.v_first_pass_yield v
  JOIN latest l
    ON to_date(concat(v.submission_month, '-01')) = l.dt
),
calc AS (
  SELECT
    date_format((SELECT dt FROM latest), 'yyyy-MM') AS submission_month,
    CASE
      WHEN submitted_claims_mo = 0 THEN 1.0
      ELSE fpy_claims_mo / submitted_claims_mo
    END AS fpy_rate,
    (SELECT fpy_target FROM params) AS target
  FROM agg
)
SELECT *
FROM calc
WHERE fpy_rate < target;